# Packages

In [1]:
# Data Processing and Analytics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
# Machine Learning and ML Preprocessing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
# Neural Networks
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
# Presets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
%matplotlib inline

## Set Seed

In [2]:
np.random.seed(100)

# Import Data

In [3]:
data = pd.read_csv('data_for_ml.csv')
data.head(2)

,INDEX_,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,3366652,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major
1,3370334,19,weekday,Rain,Dark,Wet,4.8,1.00,Midblock,Traffic Signal including Transit,Major Arterial,60,351.0,63.0,Residential,452,205,183,Over 65,Normal,Crossing without ROW,automobile,speeding,Major


In [4]:
df = data.drop('INDEX_', axis=1)
df.head(1)

,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major


In [5]:
selected_features = ['HOUR', 'TEMP', 'REL_HUMID', 'VEH_ADT', 'PED_ADT', 'POP_2021', 'PRIV_DWELL', 'LAND_AREA', 'INVAGE', 'PEDCOND', 'INJURY']
new_df = df[selected_features]
new_df.head(1)

,HOUR,TEMP,REL_HUMID,VEH_ADT,PED_ADT,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,INJURY
0,7,1.5,0.99,498.0,219.0,504,263,225,45 to 64,Distracted,Major


In [6]:
# Integer encode the INJURY column
injury_map = {
    'Fatal' : 1,
    'Major' : 0,
    'Minor' : 0,
    'Minimal' : 0,
    np.nan : 0
}
# 1 for Fatal injury and 0 for Non Fatal injury
new_df['INJURY'] = new_df['INJURY'].replace(injury_map).astype(int)
new_df.head(1)

,HOUR,TEMP,REL_HUMID,VEH_ADT,PED_ADT,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,INJURY
0,7,1.5,0.99,498.0,219.0,504,263,225,45 to 64,Distracted,0


# ML Data Preprocessing

In [7]:
# Split into X and y variables
X = new_df.iloc[:, :-1].values
y = new_df.iloc[:,-1].values

In [8]:
# One hot encode the categorical columns
cat_columns_index = [8, 9]
encoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X[:, cat_columns_index])

In [9]:
# Combine the categorical and numerical columns
num_columns_index = [0, 1, 2, 3, 4, 5, 6, 7]
X_encoded = np.concatenate((X_cat, X[:, num_columns_index].astype(float)), axis=1)
print(X_encoded)

[[0.000e+00 0.000e+00 1.000e+00 ... 5.040e+02 2.630e+02 2.250e+02]
 [0.000e+00 0.000e+00 0.000e+00 ... 4.520e+02 2.050e+02 1.830e+02]
 [1.000e+00 0.000e+00 0.000e+00 ... 8.070e+02 3.750e+02 3.220e+02]
 ...
 [0.000e+00 0.000e+00 1.000e+00 ... 4.820e+02 2.520e+02 2.310e+02]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.981e+03 1.195e+03 1.112e+03]
 [0.000e+00 0.000e+00 1.000e+00 ... 1.170e+03 4.820e+02 4.670e+02]]


In [10]:
# Split encoded X and y to train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.20, random_state=100)

In [11]:
# Scale the numerical features
scaler = StandardScaler()
X_train[:, -len(num_columns_index):] = scaler.fit_transform(X_train[:, -len(num_columns_index):])
X_test[:, -len(num_columns_index):] = scaler.transform(X_test[:, -len(num_columns_index):])
print(f"X_train dimension: {X_train.shape}")
print(f"y_train dimension: {y_train.shape}")
print(f"X_test dimension: {X_test.shape}")
print(f"y_test dimension: {y_test.shape}")

X_train dimension: (2490, 23)
y_train dimension: (2490,)
X_test dimension: (623, 23)
y_test dimension: (623,)


# ML Modeling and Metrics

In [12]:
models = [KNeighborsClassifier(), RandomForestClassifier(), XGBClassifier()]

In [13]:
# Define parameter grids
param_grid_knn = {
    'n_neighbors' : [i for i in range(1,36)],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan']
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],                
    'max_features': ['auto', 'sqrt', 'log2'],      
    'max_depth': [None, 10, 20, 30],               
    'min_samples_split': [2, 5, 10],               
    'min_samples_leaf': [1, 2, 4],                 
    'bootstrap': [True, False],                    
    'criterion': ['gini', 'entropy']               
}

param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0]
}

param_grids = [param_grid_knn, param_grid_rf, param_grid_xgb]

In [14]:
# Iterate on both grids and models to fit the model with our training data
for model, grid in zip(models, param_grids):
    random_search = RandomizedSearchCV(estimator=model,
                                      param_distributions=grid,
                                      n_iter=100,
                                      cv=5,
                                      scoring='accuracy',
                                      random_state=100)
    random_search.fit(X_train, y_train)
    y_pred = random_search.predict(X_test)
    training_accuracy = random_search.best_score_
    best_parameters = random_search.best_params_
    print(f'{model} Modeling Results:\n')
    print('Best Parameters:')
    print(f'{best_parameters}\n')
    print(f"Training Accuracy: {training_accuracy*100:.2f}%")
    print(f"Testing Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred, average='weighted')*100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred, average='weighted')*100:.2f}%")
    print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')*100:.2f}%\n")
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:")
    print(f"{cm}\n")

KNeighborsClassifier() Modeling Results:

Best Parameters:
{'weights': 'uniform', 'n_neighbors': 17, 'metric': 'euclidean'}

Training Accuracy: 83.61%
Testing Accuracy: 84.75%
Precision: 80.03%
Recall: 84.75%
F1 Score: 79.39%

Confusion Matrix:
[[522   6]
 [ 89   6]]

RandomForestClassifier() Modeling Results:

Best Parameters:
{'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'criterion': 'entropy', 'bootstrap': True}

Training Accuracy: 83.73%
Testing Accuracy: 84.27%
Precision: 78.11%
Recall: 84.27%
F1 Score: 78.86%

Confusion Matrix:
[[520   8]
 [ 90   5]]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_c

# Neural Network Modeling

In [15]:
tf.random.set_seed(100)

## Design 1 - 2 Hidden Layers

In [16]:
# Construct NN Architecture
design1 = Sequential([
    Dense(units=11, activation='relu', input_shape=(23,)),
    Dropout(rate=0.1),
    Dense(units=1, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design1.compile(loss=BinaryCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design1.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 5ms/step - loss: 0.4787 - accuracy: 0.8325 - val_loss: 0.4121 - val_accuracy: 0.8475
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4315 - accuracy: 0.8337 - val_loss: 0.4014 - val_accuracy: 0.8475
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.8337 - val_loss: 0.3988 - val_accuracy: 0.8475
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4223 - accuracy: 0.8337 - val_loss: 0.3950 - val_accuracy: 0.8475
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4207 - accuracy: 0.8337 - val_loss: 0.3921 - val_accuracy: 0.8475
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4110 - accuracy: 0.8337 - val_loss: 0.3946 - val_accuracy: 0.8475
Epoch 7/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4213 - accuracy: 0.8337 - val_loss: 0.3925 - val_accuracy: 0.8475
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3985 - accuracy: 0.8329 - val_loss: 0.3979 - val_accuracy: 0.8443
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3951 - accuracy: 0.8357 - val_loss: 0.4045 - val_accuracy: 0.8347
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3953 - accuracy: 0.8341 - val_loss: 0.3976 - val_accuracy: 0.8443
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3964 - accuracy: 0.8365 - val_loss: 0.3940 - val_accuracy: 0.8459
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3931 - accuracy: 0.8361 - val_loss: 0.3938 - val_accuracy: 0.8491
Epoch 64/100
78/78 [==============================] - 0s 3ms/step - loss: 0.3926 - accuracy: 0.8353 - val_loss: 0.3954 - val_accuracy: 0.8443
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3982 - accuracy: 0.8341 - val_loss: 0.3958 - val_accuracy: 0.8443
Epoch 

## Design 2 - 4 Hidden Layers

In [17]:
# Construct NN Architecture
design2 = Sequential([
    Dense(units=11, activation='relu', input_shape=(23,)),
    Dropout(rate=0.1),
    Dense(units=11, activation='relu'),
    Dropout(rate=0.1),
    Dense(units=1, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design2.compile(loss=BinaryCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design2.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 5ms/step - loss: 0.4728 - accuracy: 0.8337 - val_loss: 0.4089 - val_accuracy: 0.8475
Epoch 2/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4258 - accuracy: 0.8337 - val_loss: 0.3957 - val_accuracy: 0.8475
Epoch 3/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4196 - accuracy: 0.8337 - val_loss: 0.3918 - val_accuracy: 0.8475
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4199 - accuracy: 0.8337 - val_loss: 0.3888 - val_accuracy: 0.8475
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4108 - accuracy: 0.8337 - val_loss: 0.3862 - val_accuracy: 0.8475
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4059 - accuracy: 0.8337 - val_loss: 0.3955 - val_accuracy: 0.8475
Epoch 7/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4156 - accuracy: 0.8337 - val_loss: 0.3877 - val_accuracy: 0.8475
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3850 - accuracy: 0.8349 - val_loss: 0.4069 - val_accuracy: 0.8443
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3860 - accuracy: 0.8341 - val_loss: 0.4055 - val_accuracy: 0.8427
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3785 - accuracy: 0.8337 - val_loss: 0.4087 - val_accuracy: 0.8427
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3832 - accuracy: 0.8341 - val_loss: 0.3995 - val_accuracy: 0.8443
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3829 - accuracy: 0.8349 - val_loss: 0.4013 - val_accuracy: 0.8427
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3842 - accuracy: 0.8337 - val_loss: 0.4030 - val_accuracy: 0.8427
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3852 - accuracy: 0.8341 - val_loss: 0.4058 - val_accuracy: 0.8475
Epoch 

## Design 3 - 6 Hidden Layers

In [18]:
# Construct NN Architecture
design3 = Sequential([
    Dense(units=11, activation='relu', input_shape=(23,)),
    Dropout(rate=0.1),
    Dense(units=11, activation='relu'),
    Dropout(rate=0.1),
    Dense(units=11, activation='relu'),
    Dropout(rate=0.1),
    Dense(units=1, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design3.compile(loss=BinaryCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design3.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 5ms/step - loss: 0.5021 - accuracy: 0.8337 - val_loss: 0.4262 - val_accuracy: 0.8475
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4423 - accuracy: 0.8337 - val_loss: 0.3997 - val_accuracy: 0.8475
Epoch 3/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4225 - accuracy: 0.8337 - val_loss: 0.3940 - val_accuracy: 0.8475
Epoch 4/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4226 - accuracy: 0.8337 - val_loss: 0.3911 - val_accuracy: 0.8475
Epoch 5/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4158 - accuracy: 0.8337 - val_loss: 0.3896 - val_accuracy: 0.8475
Epoch 6/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4126 - accuracy: 0.8337 - val_loss: 0.3978 - val_accuracy: 0.8475
Epoch 7/100
78/78 [==============================] - 0s 3ms/step - loss: 0.4217 - accuracy: 0.8337 - val_loss: 0.3905 - val_accuracy: 0.8475
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3750 - accuracy: 0.8426 - val_loss: 0.3949 - val_accuracy: 0.8459
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3748 - accuracy: 0.8373 - val_loss: 0.4063 - val_accuracy: 0.8427
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3762 - accuracy: 0.8406 - val_loss: 0.4042 - val_accuracy: 0.8459
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3737 - accuracy: 0.8386 - val_loss: 0.3928 - val_accuracy: 0.8427
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3743 - accuracy: 0.8398 - val_loss: 0.3997 - val_accuracy: 0.8427
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3688 - accuracy: 0.8390 - val_loss: 0.3973 - val_accuracy: 0.8427
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3681 - accuracy: 0.8426 - val_loss: 0.3957 - val_accuracy: 0.8475
Epoch 

# NN Performance Metrics

In [19]:
models = [design1, design2, design3]
for model in models:
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int)
    train_loss, train_accuracy = model.evaluate(X_train, y_train)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    f1 = f1_score(y_test, y_pred_classes, average='weighted')
    print(f"Model Performance Metrics:")
    print(f"Training Accuracy: {train_accuracy*100:.2f}%")
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"Precision: {precision*100:.2f}%")
    print(f"Recall: {recall*100:.2f}%")
    print(f"F1 Score: {f1*100:.2f}%\n")
    print(f"Confusion Matrix:")
    print(f"{confusion_matrix(y_test, y_pred_classes)}\n")

20/20 [==============================] - 0s 2ms/step - loss: 0.3968 - accuracy: 0.8395
Model Performance Metrics:
Training Accuracy: 83.49%
Test Accuracy: 83.95%
Precision: 75.30%
Recall: 83.95%
F1 Score: 77.92%

Confusion Matrix:
[[521   7]
 [ 93   2]]

20/20 [==============================] - 0s 1ms/step - loss: 0.4042 - accuracy: 0.8427
Model Performance Metrics:
Training Accuracy: 83.37%
Test Accuracy: 84.27%
Precision: 71.77%
Recall: 84.27%
F1 Score: 77.52%

Confusion Matrix:
[[525   3]
 [ 95   0]]

20/20 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8459
Model Performance Metrics:
Training Accuracy: 83.98%
Test Accuracy: 84.59%
Precision: 76.98%
Recall: 84.59%
F1 Score: 77.98%

Confusion Matrix:
[[526   2]
 [ 94   1]]

